- Update: 2023.04.23

In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# = = = = = = = = = = = = = = = = = =

chromedriver = '../../chromedriver.exe'

# = = = = = = = = = = = = = = = = = =

chrome_options = ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) # Chrome正受到自动测试软件的控制
chrome_options.add_argument('--disable-blink-features=AutomationControlled') # window.navigator.webdriver=false
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
chrome_options.add_argument('--proxy-server=http://j933.kdltps.com:15818')
chrome_options.add_argument('blink-settings=imagesEnabled=false')
chrome_options.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu-LS_Oil_Pan.xlsx')
list_product = df_menu['Product'].to_list()
print('总数量：' + str(len(list_product)))
print()

# = = = = = = = = = = = = = = = = = =

df = pd.DataFrame(columns=['链接',
                           '车型',
                           'status'])

# = = = = = = = = = = = = = = = = = =

work = Queue()
for product in list_product:
    work.put_nowait(product)

# = = = = = = = = = = = = = = = = = =

def crawler():
    global df
    
    while not work.empty():
        product = work.get_nowait()
        url = product
        
        for i in range(31):
            try:
                driver = Chrome(service=Service(chromedriver), options=chrome_options)
                driver.maximize_window()
                time.sleep(0.3)
                driver.maximize_window()
                driver.get(url + '/applications')
                driver.maximize_window()
                
                time.sleep(30)
                
                # = = = = = = = = = = = = = = = = = =
                
                try:
                    driver.find_element(By.XPATH, '//*[@%clearass="columns small-24 medium-14 medium-pull-10"]')
                    df_temp = pd.DataFrame({'链接': [str(url)],
                                            '车型': [''],
                                            'status': ['ok']})
                    break
                except:
                    pass
                
                vehicle = ''
                WebDriverWait(driver, 30, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="is-accordion-submenu-parent"]')))
                list_bar = driver.find_elements(By.XPATH, '//*[@class="is-accordion-submenu-parent"]')
                for bar in list_bar:
                    if bar.find_element(By.XPATH, './a').text.strip().lower() == 'make':
                        vehicle = bar.find_element(By.XPATH, './ul').text.strip()

                # = = = = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'链接': [str(url)],
                                        '车型': [str(vehicle)],
                                        'status': ['ok']})
                
                # = = = = = = = = = = = = = = = = = =
                
                driver.close()
                
                break

            except:
                if i == 30:
                    df_temp = pd.DataFrame({'链接': [str(url)],
                                            '车型': [''],
                                            'status': ['error']})
                driver.close()
                continue

        # = = = = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
        
        print(product, ' [' + str(i) + '] - 剩余数量：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df.to_excel('./crawler-LS_Oil_Pan-vehicle.xlsx', index=False)
print()
print('搞定')

总数量：738

https://www.summitracing.com/parts/ado-11562588  [0] - 剩余数量：731
https://www.summitracing.com/parts/ado-12612350  [0] - 剩余数量：730
https://www.summitracing.com/parts/sum-csumcpgmls02  [0] - 剩余数量：729
https://www.summitracing.com/parts/ado-11570082  [1] - 剩余数量：728
https://www.summitracing.com/parts/ado-24239049  [0] - 剩余数量：727
https://www.summitracing.com/parts/ado-24203590  [0] - 剩余数量：726
https://www.summitracing.com/parts/arp-534-9605  [0] - 剩余数量：725
https://www.summitracing.com/parts/ado-pt2386  [0] - 剩余数量：724
https://www.summitracing.com/parts/arp-534-9505  [0] - 剩余数量：723
https://www.summitracing.com/parts/arp-534-9705  [0] - 剩余数量：722
https://www.summitracing.com/parts/arp-134-6902  [0] - 剩余数量：721
https://www.summitracing.com/parts/hpe-hp93c-efi  [0] - 剩余数量：720
https://www.summitracing.com/parts/hpe-ls01c  [0] - 剩余数量：719
https://www.summitracing.com/parts/atp-fg25  [0] - 剩余数量：718
https://www.summitracing.com/parts/hpe-ls02c  [0] - 剩余数量：717
https://www.summitracing.com/parts/ado

https://www.summitracing.com/parts/nal-s2022-32  [0] - 剩余数量：602
https://www.summitracing.com/parts/nal-s2022-29  [1] - 剩余数量：601
https://www.summitracing.com/parts/nal-19418256  [0] - 剩余数量：600
https://www.summitracing.com/parts/nal-12639759  [0] - 剩余数量：599
https://www.summitracing.com/parts/nal-12682080  [0] - 剩余数量：598
https://www.summitracing.com/parts/nal-12611904  [0] - 剩余数量：597
https://www.summitracing.com/parts/nal-12580855  [0] - 剩余数量：596
https://www.summitracing.com/parts/nal-12558251  [1] - 剩余数量：595
https://www.summitracing.com/parts/nal-19433039  [1] - 剩余数量：594
https://www.summitracing.com/parts/nal-19366637  [1] - 剩余数量：593
https://www.summitracing.com/parts/nal-19417102  [0] - 剩余数量：592
https://www.summitracing.com/parts/nal-24203590  [0] - 剩余数量：591
https://www.summitracing.com/parts/cvs-5414  [0] - 剩余数量：590
https://www.summitracing.com/parts/nal-s2022-63  [3] - 剩余数量：589
https://www.summitracing.com/parts/nal-19417593  [2] - 剩余数量：588
https://www.summitracing.com/parts/cvs-9314 

https://www.summitracing.com/parts/hed-45696  [1] - 剩余数量：472
https://www.summitracing.com/parts/hly-302-10  [0] - 剩余数量：471
https://www.summitracing.com/parts/hly-302-11  [0] - 剩余数量：470
https://www.summitracing.com/parts/hok-71222016hkr  [0] - 剩余数量：469
https://www.summitracing.com/parts/hly-302-12  [0] - 剩余数量：468
https://www.summitracing.com/parts/hok-71222012hkr  [0] - 剩余数量：467
https://www.summitracing.com/parts/hok-71222017hkr  [1] - 剩余数量：466
https://www.summitracing.com/parts/hok-71222024hkr1  [0] - 剩余数量：465
https://www.summitracing.com/parts/hok-71223013hkr  [0] - 剩余数量：464
https://www.summitracing.com/parts/hok-bhs525  [0] - 剩余数量：463
https://www.summitracing.com/parts/hok-bhs507  [0] - 剩余数量：462
https://www.summitracing.com/parts/hok-bhs586  [1] - 剩余数量：461
https://www.summitracing.com/parts/hok-71222020hkr  [0] - 剩余数量：460
https://www.summitracing.com/parts/hok-71222013hkr  [0] - 剩余数量：459
https://www.summitracing.com/parts/hok-bhs550  [0] - 剩余数量：458
https://www.summitracing.com/parts/

https://www.summitracing.com/parts/sum-csummrbbp003  [0] - 剩余数量：343
https://www.summitracing.com/parts/mml-oslsx401120  [1] - 剩余数量：342
https://www.summitracing.com/parts/mil-31063  [0] - 剩余数量：341
https://www.summitracing.com/parts/mor-21149  [0] - 剩余数量：340
https://www.summitracing.com/parts/mil-30914  [1] - 剩余数量：339
https://www.summitracing.com/parts/mil-30915  [2] - 剩余数量：338
https://www.summitracing.com/parts/mor-20152  [0] - 剩余数量：337
https://www.summitracing.com/parts/mor-21153  [1] - 剩余数量：336
https://www.summitracing.com/parts/mor-20148  [0] - 剩余数量：335
https://www.summitracing.com/parts/mor-20144  [0] - 剩余数量：334
https://www.summitracing.com/parts/mor-21172  [0] - 剩余数量：333
https://www.summitracing.com/parts/mor-21157  [1] - 剩余数量：332
https://www.summitracing.com/parts/mor-20149  [0] - 剩余数量：331
https://www.summitracing.com/parts/mor-21159  [0] - 剩余数量：330
https://www.summitracing.com/parts/mor-23707  [0] - 剩余数量：329
https://www.summitracing.com/parts/mor-71162  [0] - 剩余数量：328
https://www

https://www.summitracing.com/parts/cmb-08-0149  [0] - 剩余数量：212
https://www.summitracing.com/parts/cmb-08-0150  [0] - 剩余数量：211
https://www.summitracing.com/parts/cmb-08-0151  [0] - 剩余数量：210
https://www.summitracing.com/parts/mtl-108212  [30] - 剩余数量：209
https://www.summitracing.com/parts/cmb-08-0156  [0] - 剩余数量：208
https://www.summitracing.com/parts/mtl-109471  [30] - 剩余数量：207
https://www.summitracing.com/parts/cmb-08-0160  [0] - 剩余数量：206
https://www.summitracing.com/parts/cmb-08-0161  [1] - 剩余数量：205
https://www.summitracing.com/parts/cmb-08-0166  [0] - 剩余数量：204
https://www.summitracing.com/parts/cmb-08-0170  [0] - 剩余数量：203
https://www.summitracing.com/parts/cmb-08-0165  [0] - 剩余数量：202
https://www.summitracing.com/parts/cmb-08-0171  [0] - 剩余数量：201
https://www.summitracing.com/parts/cmb-08-0155  [0] - 剩余数量：200
https://www.summitracing.com/parts/cmb-08-0175  [0] - 剩余数量：199
https://www.summitracing.com/parts/cmb-08-0180  [0] - 剩余数量：198
https://www.summitracing.com/parts/cmb-08-0181  [0] - 剩

https://www.summitracing.com/parts/trd-48062  [0] - 剩余数量：79
https://www.summitracing.com/parts/trd-42225  [1] - 剩余数量：78
https://www.summitracing.com/parts/trd-48063  [0] - 剩余数量：77
https://www.summitracing.com/parts/trd-46001  [0] - 剩余数量：76
https://www.summitracing.com/parts/trd-46004  [0] - 剩余数量：75
https://www.summitracing.com/parts/trd-46007  [0] - 剩余数量：74
https://www.summitracing.com/parts/trd-46005  [0] - 剩余数量：73
https://www.summitracing.com/parts/trd-42242  [0] - 剩余数量：72
https://www.summitracing.com/parts/trd-46008  [1] - 剩余数量：71
https://www.summitracing.com/parts/trd-46011  [0] - 剩余数量：70
https://www.summitracing.com/parts/trd-46014  [0] - 剩余数量：69
https://www.summitracing.com/parts/trd-46015  [0] - 剩余数量：68
https://www.summitracing.com/parts/trd-46018  [0] - 剩余数量：67
https://www.summitracing.com/parts/trd-46012  [0] - 剩余数量：66
https://www.summitracing.com/parts/trd-42921  [0] - 剩余数量：65
https://www.summitracing.com/parts/trd-42214  [0] - 剩余数量：64
https://www.summitracing.com/parts/trd-4

D:\Lennon\python\venv\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
